<h1>Código para extraer estadísticas generacionales de Oaxaca<h1/>

In [21]:
import pandas as pd

# Definir las rutas de los archivos
archivos = {
    "1990": "C:/Users/quesa/Downloads/ITER_20XLS90.xls",
    "1995": "C:/Users/quesa/Downloads/ITER_20XLS95.xls",
    "2000": "C:/Users/quesa/Downloads/cgpv2000_iter_20.csv",
    "2005": "C:/Users/quesa/Downloads/cpv2005_iter_20.csv",
    "2010": "C:/Users/quesa/Downloads/iter_20_cpv2010.csv",
    "2020": "C:/Users/quesa/Downloads/conjunto_de_datos_iter_20CSV20.csv",
}
archivo_claves = "C:/Users/quesa/Desktop/Ejercicio/ClavesMetro.csv"

# Leer las claves de municipios de la zona metropolitana y filtrar los que tienen 'Oaxaca' en NOM_MET
claves_metro = pd.read_csv(archivo_claves)
claves_metro = claves_metro[claves_metro['NOM_MET'].str.contains('Oaxaca', case=False, na=False)]
claves_metro['CVEMUN'] = claves_metro['CVEMUN'].astype(str)

# Diccionario de correspondencias entre columnas
columnas_datos = {
    "1990": {"poblacion": "P_TOTAL\t", "viviendas": "T_VIVHAB\t", "ent": "ENTIDAD", "mun": "MUN", "loc": "LOC\t", "nom_mun": "NOM_MUN\t"},
    "1995": {"poblacion": "POBTOTAL", "viviendas": "TOTVIVH", "ent": "ENTIDAD", "mun": "MUN", "loc": "LOC\t", "nom_mun": "NOM_MUN\t"},
    "2000": {"poblacion": "pobtot", "viviendas": "totvivhab", "ent": "entidad", "mun": "mun", "loc": "loc", "nom_mun": "nom_mun"},
    "2005": {"poblacion": "p_total", "viviendas": "t_vivhab", "ent": "entidad", "mun": "mun", "loc": "loc", "nom_mun": "nom_mun"},
    "2010": {"poblacion": "pobtot", "viviendas": "vivtot", "ent": "entidad", "mun": "mun", "loc": "loc", "nom_mun": "nom_mun"},
    "2020": {"poblacion": "POBTOT", "viviendas": "VIVTOT", "ent": "ENTIDAD", "mun": "MUN", "loc": "LOC", "nom_mun": "NOM_MUN"},
}

# Crear un DataFrame vacío para almacenar los resultados combinados
df_combinado = pd.DataFrame()

# Procesar los datos de cada año
for anio, ruta in archivos.items():
    # Identificar formato del archivo
    if ruta.endswith(".xls"):
        df = pd.read_excel(ruta)
    elif ruta.endswith(".csv"):
        df = pd.read_csv(ruta)
    
    # Crear la columna CVEMUN para hacer el join
    columnas = columnas_datos[anio]
    df['ENTIDAD'] = df[columnas['ent']].astype(str).str.zfill(2)
    df['MUN'] = df[columnas['mun']].astype(str).str.zfill(3)
    df['CVEMUN'] = df['ENTIDAD'] + df['MUN']
    
    # Filtrar solo municipios de la zona metropolitana con 'Oaxaca' en NOM_MET
    df_metro = df[df['CVEMUN'].isin(claves_metro['CVEMUN'])]
    
    # Filtrar por la columna de localidad (loc) donde sea '0000'
    df_metro = df_metro[df_metro[columnas['loc']] == 0]
    
    # Seleccionar columnas relevantes, incluyendo NOM_MUN
    df_metro = df_metro[['CVEMUN', columnas['nom_mun'], columnas['poblacion'], columnas['viviendas']]].rename(
        columns={
            columnas['nom_mun']: "Nombre_Municipio",
            columnas['poblacion']: f"{anio}_pob",
            columnas['viviendas']: f"{anio}_viv"
        }
    )
    
    # Unir los datos del año actual con el DataFrame combinado
    if df_combinado.empty:
        df_combinado = df_metro
    else:
        df_combinado = df_combinado.merge(df_metro, on=['CVEMUN', 'Nombre_Municipio'], how='outer')

# Exportar los datos combinados a un archivo Excel
output_file = "C:/Users/quesa/Desktop/Zona_Metro_Oaxaca.xlsx"
df_combinado.to_excel(output_file, index=False)

print(f"Los datos han sido exportados a: {output_file}")

C:\Users\quesa\AppData\Local\Temp\ipykernel_6348\2002227179.py:38: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta)
C:\Users\quesa\AppData\Local\Temp\ipykernel_6348\2002227179.py:38: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta)


Los datos han sido exportados a: C:/Users/quesa/Desktop/Zona_Metro_Oaxaca.xlsx
